In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
import csv

In [69]:
import re

In [3]:
# from pandas.io.json import json_normalize

In [3]:
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)# Установите для отображения самой большой линии

In [4]:
list_columns = [
    'experience',
    'specializations',
    'key_skills'
]

In [5]:
df = pd.read_csv('region_roles_26.05.22_CSV.csv', usecols=list_columns, encoding='windows-1251', sep=';')

In [6]:
df.head(2)

,experience,key_skills,specializations
0,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Информационная безопасность'}, {'na...","[{'id': '1.110', 'name': 'Компьютерная безопас..."
1,"{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","[{'name': 'Информационная безопасность'}, {'na...","[{'id': '1.110', 'name': 'Компьютерная безопас..."


In [7]:
# -> Пишем функцию для будущего apply
# Добавляем try except для отлова проблемных текстов и удаление всех двойных кавычек

def text_to_json(text):
    try:
        return json.loads(json.dumps(text)).replace("'", '"')
    except:
        print('Ошибка: ', text)
        # return []

In [8]:
df.specializations = df.specializations.fillna('[]')

In [10]:
df['json_specializations'] = df.specializations.apply(text_to_json)

In [11]:
t = df['json_specializations'][1]

In [12]:
t

'[{"id": "1.110", "name": "Компьютерная безопасность", "profarea_id": "1", "profarea_name": "Информационные технологии, интернет, телеком"}, {"id": "1.246", "name": "Развитие бизнеса", "profarea_id": "1", "profarea_name": "Информационные технологии, интернет, телеком"}, {"id": "1.25", "name": "Аналитик", "profarea_id": "1", "profarea_name": "Информационные технологии, интернет, телеком"}, {"id": "3.98", "name": "Исследования рынка", "profarea_id": "3", "profarea_name": "Маркетинг, реклама, PR"}, {"id": "3.150", "name": "Менеджмент продукта (Product manager)", "profarea_id": "3", "profarea_name": "Маркетинг, реклама, PR"}, {"id": "8.356", "name": "Экономическая и информационная безопасность", "profarea_id": "8", "profarea_name": "Безопасность"}]'

In [13]:
t_list = json.loads(t)

In [20]:
t_list

[{'id': '1.110',
  'name': 'Компьютерная безопасность',
  'profarea_id': '1',
  'profarea_name': 'Информационные технологии, интернет, телеком'},
 {'id': '1.246',
  'name': 'Развитие бизнеса',
  'profarea_id': '1',
  'profarea_name': 'Информационные технологии, интернет, телеком'},
 {'id': '1.25',
  'name': 'Аналитик',
  'profarea_id': '1',
  'profarea_name': 'Информационные технологии, интернет, телеком'},
 {'id': '3.98',
  'name': 'Исследования рынка',
  'profarea_id': '3',
  'profarea_name': 'Маркетинг, реклама, PR'},
 {'id': '3.150',
  'name': 'Менеджмент продукта (Product manager)',
  'profarea_id': '3',
  'profarea_name': 'Маркетинг, реклама, PR'},
 {'id': '8.356',
  'name': 'Экономическая и информационная безопасность',
  'profarea_id': '8',
  'profarea_name': 'Безопасность'}]

In [81]:
res = pd.json_normalize(t_list, max_level=0).sort_values(by='id')

In [85]:
res

,id,name,profarea_id,profarea_name
0,1.110,Компьютерная безопасность,1,"Информационные технологии, интернет, телеком"
1,1.246,Развитие бизнеса,1,"Информационные технологии, интернет, телеком"
2,1.250,Аналитик,1,"Информационные технологии, интернет, телеком"
4,3.150,Менеджмент продукта (Product manager),3,"Маркетинг, реклама, PR"
3,3.980,Исследования рынка,3,"Маркетинг, реклама, PR"
5,8.356,Экономическая и информационная безопасность,8,Безопасность


In [83]:
res['id'] = pd.to_numeric(res['id'])

In [ ]:
res.info()

In [86]:
lst_1 = []
for i in range(len(res)):
    for column in res.columns:
        lst_1.append(column + '_' + str(i))
print(lst_1)

['id_0', 'name_0', 'profarea_id_0', 'profarea_name_0', 'id_1', 'name_1', 'profarea_id_1', 'profarea_name_1', 'id_2', 'name_2', 'profarea_id_2', 'profarea_name_2', 'id_3', 'name_3', 'profarea_id_3', 'profarea_name_3', 'id_4', 'name_4', 'profarea_id_4', 'profarea_name_4', 'id_5', 'name_5', 'profarea_id_5', 'profarea_name_5']


In [87]:
lst_2 = []
for index in res.index:
    lst_2.extend(list(res.loc[index,:]))
print(lst_2)

[1.11, 'Компьютерная безопасность', '1', 'Информационные технологии, интернет, телеком', 1.246, 'Развитие бизнеса', '1', 'Информационные технологии, интернет, телеком', 1.25, 'Аналитик', '1', 'Информационные технологии, интернет, телеком', 3.15, 'Менеджмент продукта (Product manager)', '3', 'Маркетинг, реклама, PR', 3.98, 'Исследования рынка', '3', 'Маркетинг, реклама, PR', 8.356, 'Экономическая и информационная безопасность', '8', 'Безопасность']


In [88]:
tf_final = pd.DataFrame(columns=lst_1)
tf_final.loc[0] = lst_2
tf_final

,id_0,name_0,profarea_id_0,profarea_name_0,id_1,name_1,profarea_id_1,profarea_name_1,id_2,name_2,...,profarea_id_3,profarea_name_3,id_4,name_4,profarea_id_4,profarea_name_4,id_5,name_5,profarea_id_5,profarea_name_5
0,1.11,Компьютерная безопасность,1,"Информационные технологии, интернет, телеком",1.246,Развитие бизнеса,1,"Информационные технологии, интернет, телеком",1.25,Аналитик,...,3,"Маркетинг, реклама, PR",3.98,Исследования рынка,3,"Маркетинг, реклама, PR",8.356,Экономическая и информационная безопасность,8,Безопасность


In [36]:
tf = pd.DataFrame({'A': list(range(1,5)), 'B': list(range(10,50,10))})

In [37]:
tf

,A,B
0,1,10
1,2,20
2,3,30
3,4,40


In [41]:
lst_1 = []
for i in range(len(tf)):
    for column in tf.columns:
        lst_1.append(column + '_' + str(i))
print(lst_1)

['A_0', 'B_0', 'A_1', 'B_1', 'A_2', 'B_2', 'A_3', 'B_3']


In [42]:
lst_2 = []
for index in tf.index:
    lst_2.extend(list(tf.loc[index,:]))
print(lst_2)

[1, 10, 2, 20, 3, 30, 4, 40]


In [43]:
tf_final = pd.DataFrame(columns=lst_1)
tf_final.loc[0] = lst_2
tf_final

,A_0,B_0,A_1,B_1,A_2,B_2,A_3,B_3
0,1,10,2,20,3,30,4,40


In [44]:
tr = [] # для списка значений
tr_out_cols = [] # для названий столбцов новой таблицы

for c in tf.columns:
    tr_out_cols = tr_out_cols + list(c+'_'+str(m) for m in range(1,tf.shape[0]+1))
    tr = tr + tf[c].to_list()

ltf = pd.DataFrame( [tr], columns= tr_out_cols )

In [45]:
ltf

,A_1,A_2,A_3,A_4,B_1,B_2,B_3,B_4
0,1,2,3,4,10,20,30,40
